In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import os

# import sys

# librosa is a Python library for analyzing audio and music. 
#It can be used to extract the data from the audio files we will see it later.
# import librosa
# import librosa.display


# from sklearn.preprocessing import StandardScaler, OneHotEncoder

# to play the audio files
import IPython.display as ipd



In [2]:
"""#Paths listed out


#Bea - Female
"/EmoV-DB_sorted/bea/Amused"
"/EmoV-DB_sorted/bea/Angry"
"/EmoV-DB_sorted/bea/Disgusted"
"/EmoV-DB_sorted/bea/Neutral"
"/EmoV-DB_sorted/bea/Sleepy"

#Jenie - Female
"/EmoV-DB_sorted/jenie/Amused"
"/EmoV-DB_sorted/jenie/Angry"
"/EmoV-DB_sorted/jenie/Disgusted"
"/EmoV-DB_sorted/jenie/Neutral"
"/EmoV-DB_sorted/jenie/Sleepy"


#Josh - Male
"/EmoV-DB_sorted/josh/Amused"
"/EmoV-DB_sorted/josh/Neutral"
"/EmoV-DB_sorted/josh/Sleepy"


#Sam - Male
"/EmoV-DB_sorted/sam/Amused"
"/EmoV-DB_sorted/sam/Angry"
"/EmoV-DB_sorted/sam/Disgusted"
"/EmoV-DB_sorted/sam/Neutral"
"/EmoV-DB_sorted/sam/Sleepy"
"""

'#Paths listed out\n\n\n#Bea - Female\n"/EmoV-DB_sorted/bea/Amused"\n"/EmoV-DB_sorted/bea/Angry"\n"/EmoV-DB_sorted/bea/Disgusted"\n"/EmoV-DB_sorted/bea/Neutral"\n"/EmoV-DB_sorted/bea/Sleepy"\n\n#Jenie - Female\n"/EmoV-DB_sorted/jenie/Amused"\n"/EmoV-DB_sorted/jenie/Angry"\n"/EmoV-DB_sorted/jenie/Disgusted"\n"/EmoV-DB_sorted/jenie/Neutral"\n"/EmoV-DB_sorted/jenie/Sleepy"\n\n\n#Josh - Male\n"/EmoV-DB_sorted/josh/Amused"\n"/EmoV-DB_sorted/josh/Neutral"\n"/EmoV-DB_sorted/josh/Sleepy"\n\n\n#Sam - Male\n"/EmoV-DB_sorted/sam/Amused"\n"/EmoV-DB_sorted/sam/Angry"\n"/EmoV-DB_sorted/sam/Disgusted"\n"/EmoV-DB_sorted/sam/Neutral"\n"/EmoV-DB_sorted/sam/Sleepy"\n'

In [3]:
#Pandas dictoionary
#Index|File name| Actor Name| Male or Female| Emotion Expressed|full file path
    #Tag with Actor name and male/ female from folder
    #Tag with emotion expressed based on sub folder


In [4]:
# Should I then  go through data and make it noisier/pitch up and down/ speed up and down to add 
# extra data to help with training?

In [5]:
#Once df completed 
        #go through each file path and for every WAV file make into spectogram for analysis??
        #go through each file path and for every WAV file create time series and a waveform graph??
        
        
        


In [6]:
#once We have all data and its been combined with artifical data do some low hanging fruit models
#Bar graph of percent of data that is each emotion
#Male vs female
#Each actor's percent of total
#each actor's percent of each emotion

In [7]:
#Perform test train split - .25% test

In [8]:
#Consider switch to google colab for processing power

#Baseline Idea.  if line/waveform do multinomnb
# images unforuntately have to do a cnn